# Worksheet 08

Name: Jonathan Thomas
UID: U81823882

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [6]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.348580671096639, 5.290230993770903, 4.679230975450958, 5.908087277635816, 3.2243512064512245, 5.364669406169521, 5.149829300951774, 4.0321081583736635, 4.164078183425797, 4.597644184370205]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [7]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[8.800154426570247, 8.161643879842266, 9.019508919867487, 7.336060282674813, 5.97633745957963, 8.206572056674291, 7.033693950593069, 6.253483862045935, 8.88535398175128, 7.920539113256612]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [8]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[7.920539113256612, 4.597644184370205, 4.164078183425797, 8.88535398175128, 4.0321081583736635, 6.253483862045935, 7.033693950593069, 5.149829300951774, 5.364669406169521, 3.2243512064512245]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

- Means and Variances of each species distribution (s1 ~ Normal(10, 1) & s2 ~ Normal(8,1))
- Prior probabilites of each species (coin flip/0.5 for both)


d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [9]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.597644184370205, 4.164078183425797, 4.0321081583736635, 6.253483862045935, 5.149829300951774, 5.364669406169521, 3.2243512064512245]
[7.920539113256612, 8.88535398175128, 7.033693950593069]
P(S_1) = 0.7,  P(S_2) = 0.3
mean_1 = 4.683737757398303,  mean_2 = 7.94652901520032
var_1 = 0.8538429265535736,  var_2 = 0.5717785493329929


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [10]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  7.920539113256612
probability of observing that point if it came from cluster 0 =  0.0003539868654053757
probability of observing that point if it came from cluster 1 =  0.6970012502793519
point =  4.597644184370205
probability of observing that point if it came from cluster 0 =  0.46486235937626136
probability of observing that point if it came from cluster 1 =  2.4812139564433036e-08
point =  4.164078183425797
probability of observing that point if it came from cluster 0 =  0.38823860763955065
probability of observing that point if it came from cluster 1 =  2.192918102630764e-10
point =  8.88535398175128
probability of observing that point if it came from cluster 0 =  2.578722631575194e-06
probability of observing that point if it came from cluster 1 =  0.1812432316596433
point =  4.0321081583736635
probability of observing that point if it came from cluster 0 =  0.34918731621500304
probability of observing that point if it came from cluster 1 =  4.638285409381032e-11
point 

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [15]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [sum([(x[0] * x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([(x[0] * x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.7,  P(S_2) = 0.3
mean_1 = 4.712431094534792,  mean_2 = 7.9575463148811
var_1 = 9.184724319521532,  var_2 = 151.30013331617846


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)